In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

RAW_DIR = Path("..") / "Data"
PROCESSED_DIR = Path("..") / "data" / "processed"

PRICE_SCALE = 10000.0
LEVELS = 10
ASK_DUMMY = 9999999999
BID_DUMMY = -9999999999

MSG_PATH = RAW_DIR / "GOOG_2012-06-21_34200000_57600000_message_10.csv"
OB_PATH  = RAW_DIR / "GOOG_2012-06-21_34200000_57600000_orderbook_10.csv"

PARQUET_PATH = PROCESSED_DIR / "goog_2012-06-21.parquet"
PICKLE_PATH  = PROCESSED_DIR / "goog_2012-06-21.pkl"


def load_df_standalone(levels: int = LEVELS) -> pd.DataFrame:
    if PARQUET_PATH.exists():
        return pd.read_parquet(PARQUET_PATH)
    if PICKLE_PATH.exists():
        return pd.read_pickle(PICKLE_PATH)

    msg_cols = ["time", "type", "order_id", "size", "price", "direction"]
    msg = pd.read_csv(MSG_PATH, header=None, names=msg_cols)

    ob_cols = []
    for lvl in range(1, levels + 1):
        ob_cols += [f"ask_p{lvl}", f"ask_s{lvl}", f"bid_p{lvl}", f"bid_s{lvl}"]
    ob = pd.read_csv(OB_PATH, header=None, names=ob_cols, usecols=list(range(4 * levels)))

    assert len(msg) == len(ob), f"Row mismatch: msg={len(msg)} vs ob={len(ob)}"

    df = pd.concat([msg, ob], axis=1).sort_values("time", kind="mergesort")
    df = df.set_index("time", drop=False)

    ask_price_cols = [f"ask_p{i}" for i in range(1, levels + 1)]
    bid_price_cols = [f"bid_p{i}" for i in range(1, levels + 1)]
    df[ask_price_cols] = df[ask_price_cols].replace(ASK_DUMMY, np.nan)
    df[bid_price_cols] = df[bid_price_cols].replace(BID_DUMMY, np.nan)

    return df


df = load_df_standalone()
df.head()


In [ ]:
valid_top = df["ask_p1"].notna() & df["bid_p1"].notna()

df["mid_d"] = np.nan
df["spread_d"] = np.nan

df.loc[valid_top, "mid_d"] = (df.loc[valid_top, "ask_p1"] + df.loc[valid_top, "bid_p1"]) / (2 * PRICE_SCALE)
df.loc[valid_top, "spread_d"] = (df.loc[valid_top, "ask_p1"] - df.loc[valid_top, "bid_p1"]) / PRICE_SCALE

den = (df["bid_s1"] + df["ask_s1"]).replace(0, np.nan)

df["micro_d"] = np.nan
df["imbalance_l1"] = np.nan
df.loc[valid_top, "micro_d"] = (
    (df.loc[valid_top, "ask_p1"] * df.loc[valid_top, "bid_s1"] + df.loc[valid_top, "bid_p1"] * df.loc[valid_top, "ask_s1"])
    / den.loc[valid_top]
) / PRICE_SCALE
df.loc[valid_top, "imbalance_l1"] = (df.loc[valid_top, "bid_s1"] - df.loc[valid_top, "ask_s1"]) / den.loc[valid_top]

# sanity
neg_share = (df.loc[valid_top, "spread_d"] < 0).mean()
assert neg_share < 1e-6, f"Negative spread share too high: {neg_share:.2e}"

df[["mid_d", "spread_d", "micro_d", "imbalance_l1"]].describe()


In [ ]:
def run_mm_type4_with_quoting_rule(
    df,
    base_size=10,
    fee_per_share=0.0,
    inv_cap=200,
    refresh_events=10,
    min_spread_d=0.01,
    latency_events=0,
    inv_scale=200,   # aggressive size-skew
    min_size=1.0     # avoid going to zero size
):
    """
    Event-driven market maker using LOBSTER visible executions (type=4) with quoting rule + latency + size-skew.

    Quoting:
      - Quotes persist and are refreshed every `refresh_events` events.
      - At refresh times, decide whether to quote based on:
          * min spread filter (profitability proxy)
          * inventory-based SIZE skew (continuous inventory control)
          * hard inventory cap safety

    Fills:
      - Only on type=4 events.
      - Buy fill if direction==1 and exec price equals our quoted bid AND we are quoting bid.
      - Sell fill if direction==-1 and exec price equals our quoted ask AND we are quoting ask.
      - Fill size uses a pro-rata approximation vs L1 depth (no queue position).

    Latency:
      - `latency_events` uses stale book info when setting quotes (0 means use i-1).
    """
    n = len(df)

    bid_p = df["bid_p1"].to_numpy()
    ask_p = df["ask_p1"].to_numpy()
    bid_q = df["bid_s1"].to_numpy()
    ask_q = df["ask_s1"].to_numpy()

    m_type  = df["type"].to_numpy()
    m_price = df["price"].to_numpy()
    m_size  = df["size"].to_numpy()
    m_dir   = df["direction"].to_numpy()

    mid_d    = df["mid_d"].to_numpy()
    spread_d = df["spread_d"].to_numpy()

    inv = np.zeros(n, dtype=float)
    cash = np.zeros(n, dtype=float)
    buys = np.zeros(n, dtype=float)
    sells = np.zeros(n, dtype=float)

    cur_inv = 0.0
    cur_cash = 0.0

    # current (resting) quote state
    q_bid = np.nan
    q_ask = np.nan
    quote_bid = False
    quote_ask = False

    # size posted on each side (can be skewed)
    q_size_bid = 0.0
    q_size_ask = 0.0


    # start index must allow i-1-latency_events to be valid
    start = 1 + latency_events

    for i in range(start, n):

        #refresh quotes every `refresh_events` events
        if (i - start) % refresh_events == 0:
            j = i - 1 - latency_events  # snapshot index used to set quotes

            # profitability filter: if spread too tight, quote nothing
            if spread_d[j] < min_spread_d:
                quote_bid = False
                quote_ask = False
                q_bid = np.nan
                q_ask = np.nan
                q_size_bid = 0.0
                q_size_ask = 0.0
            else:
                # quote at touch using stale snapshot j
                q_bid = bid_p[j]
                q_ask = ask_p[j]

                # quote both sides by default
                quote_bid = True
                quote_ask = True

                #inventory-based SIZE skew
                # k in [-1, 1]; long => k>0 => bid size smaller, ask size larger
                k = float(np.clip(cur_inv / inv_scale, -1.0, 1.0))
                q_size_bid = max(min_size, base_size * (1.0 - k))
                q_size_ask = max(min_size, base_size * (1.0 + k))

                #hard safety cap: turn off side that worsens risk
                if cur_inv >= inv_cap:
                    quote_bid = False
                    q_size_bid = 0.0
                if cur_inv <= -inv_cap:
                    quote_ask = False
                    q_size_ask = 0.0

        #fills happen only if we are quoting that side
        if m_type[i] == 4:
            # BUY fill on bid (direction==1 => execution at bid side)
            if quote_bid and (m_dir[i] == 1) and (m_price[i] == q_bid) and (bid_q[i-1] > 0):
                pre_depth = bid_q[i-1]
                exec_sz = m_size[i]
                fill = min(q_size_bid, exec_sz * (q_size_bid / pre_depth))
                px = q_bid / PRICE_SCALE

                cur_inv += fill
                cur_cash -= fill * px
                cur_cash -= fee_per_share * fill
                buys[i] = fill

            # SELL fill on ask (direction==-1 => execution at ask side)
            if quote_ask and (m_dir[i] == -1) and (m_price[i] == q_ask) and (ask_q[i-1] > 0):
                pre_depth = ask_q[i-1]
                exec_sz = m_size[i]
                fill = min(q_size_ask, exec_sz * (q_size_ask / pre_depth))
                px = q_ask / PRICE_SCALE

                cur_inv -= fill
                cur_cash += fill * px
                cur_cash -= fee_per_share * fill
                sells[i] = fill

        inv[i] = cur_inv
        cash[i] = cur_cash


    equity = cash + inv * mid_d

    return pd.DataFrame(
        {"inventory": inv, "cash": cash, "equity": equity, "buys": buys, "sells": sells},
        index=df.index
    )




In [ ]:
res_quote = run_mm_type4_with_quoting_rule(
    df,
    base_size=10,
    fee_per_share=0.0005, 
    inv_cap=200,
    refresh_events=10,
    min_spread_d=0.01,
    latency_events=10,   
    inv_scale=100,   
    min_size=1.0
)

print("Total buys:", res_quote["buys"].sum())
print("Total sells:", res_quote["sells"].sum())
print("Final equity:", res_quote["equity"].iloc[-1])
print("Max abs inventory:", res_quote["inventory"].abs().max())




In [ ]:
def plot_equity_inventory(res: pd.DataFrame, title_prefix: str, N: int = 200000):
    idx = res.index[:N]

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(idx, res["equity"].iloc[:N].values, linewidth=1)
    ax.set_title(f"{title_prefix} — Equity (PnL)")
    ax.set_xlabel("time (seconds since midnight)")
    ax.set_ylabel("equity ($)")
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(idx, res["inventory"].iloc[:N].values, linewidth=1)
    ax.set_title(f"{title_prefix} — Inventory")
    ax.set_xlabel("time (seconds since midnight)")
    ax.set_ylabel("shares")
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    plt.show()



In [ ]:
def compute_metrics(res: pd.DataFrame) -> dict:
    equity = res["equity"].to_numpy()
    inv = res["inventory"].to_numpy()

    # max drawdown (on equity curve)
    running_max = np.maximum.accumulate(equity)
    drawdown = equity - running_max
    max_dd = drawdown.min()

    return {
        "final_equity": float(equity[-1]),
        "max_drawdown": float(max_dd),
        "max_abs_inventory": float(np.max(np.abs(inv))),
        "total_buys": float(res["buys"].sum()),
        "total_sells": float(res["sells"].sum()),
        "total_trades": float(res["buys"].sum() + res["sells"].sum()),
    }


In [ ]:
# Base parameters (keep these fixed for fair comparison)
base_params = dict(
    base_size=10,
    inv_cap=200,
    refresh_events=10,
    min_spread_d=0.01,
    fee_per_share=0.0,
    inv_scale=200,   # aggressive size-skew
    min_size=1.0
)

latencies = [0, 5, 10, 20, 50]

rows = []
results = {}  # store full result dfs if you want to plot later

for L in latencies:
    res_L = run_mm_type4_with_quoting_rule(df, latency_events=L, **base_params)
    results[L] = res_L

    m = compute_metrics(res_L)
    m["latency_events"] = L
    rows.append(m)

summary = pd.DataFrame(rows).sort_values("latency_events")
summary


In [ ]:
def plot_equity_inventory(res: pd.DataFrame, title_prefix: str, N: int = 200000):
    # Equity
    plt.figure(figsize=(10,4))
    res["equity"].iloc[:N].plot()
    plt.title(f"{title_prefix} - Equity (PnL)")
    plt.xlabel("time(seconds)")
    plt.ylabel("equity ($)")
    plt.tight_layout()
    plt.show()

    # Inventory
    plt.figure(figsize=(10,4))
    res["inventory"].iloc[:N].plot()
    plt.title(f"{title_prefix} - Inventory")
    plt.xlabel("time(seconds)")
    plt.ylabel("shares")
    plt.tight_layout()
    plt.show()

plot_equity_inventory(results[0], "Latency 0", N=200000)
plot_equity_inventory(results[50], "Latency 50", N=200000)


In [ ]:
# Ablation: disable inventory-based skew by setting inv_scale very large (zero sensitivity)
params_no_skew = dict(base_params)
params_no_skew["inv_scale"] = 1_000_000

res_no_skew = run_mm_type4_with_quoting_rule(df, latency_events=50, **params_no_skew)
res_skew    = run_mm_type4_with_quoting_rule(df, latency_events=50, **base_params)

summary = pd.DataFrame([
    {"policy": "no_skew",   **compute_metrics(res_no_skew)},
    {"policy": "size_skew", **compute_metrics(res_skew)},
])

summary = summary.set_index("policy")
summary